In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import timeit
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10,15)
ds = sio.loadmat('face.mat')

In [2]:
X = ds['X']
l = ds['l'][0]
split = 0.8
W = 46
H = 56
IDs = 52
X = pd.DataFrame(X.transpose())
l = pd.DataFrame(l)

#bagging 
bag_ratio = 0.75
M_pca_ratio = 0.95
M_lda_ratio = 0.90
N = round(X.shape[0]*split) 
N_bag = round(X.shape[0]*split*bag_ratio) #different for each bag now

In [3]:
X_train, X_test, l_train, l_test = train_test_split(X, l, test_size=(1-split), stratify = l)
# X_bar = np.mean(X, axis=1)
X_train, X_test = X_train.values, X_test.values
l_train, l_test = l_train.values, np.transpose(l_test.values)
X_bar = np.transpose([np.mean(X_train, axis=0)])
X_bar = np.ndarray.flatten(X_bar)
print('    l_train.shape',l_train.shape)
print('    X_train.shape', X_train.shape)

    l_train.shape (416, 1)
    X_train.shape (416, 2576)


## Functions to apply 1.3_a to bagging

In [4]:
from random import randrange
#Take the current split data and bag randomly 
def make_bag(X_train_data, l_train_data):
    X_train_part = bag_train_space(X_train_data, l_train_data, 1)
    X_bag_part = []
    l_bag_part = []
#     print(X_train.shape)
#     print(bag, bag_size)
    for i in range(IDs):
#         print('Bullshit ' , len(X_train[i]))
        for j in range(round(len(X_train_part[i])*bag_ratio)):
            index = randrange(len(X_train_part[i]))
#             print(index)
            X_bag_part.append(X_train_part[i][index])
            l_bag_part.append(i+1)
    X_bar_part_bag = np.transpose([np.mean(X_bag_part, axis=0)])
    X_bar_part_bag= np.ndarray.flatten(X_bar_part_bag)
#     print()
    return np.asarray(X_bag_part), np.asarray(l_bag_part), np.asarray(X_bar_part_bag)

In [5]:
def bag_train_space(X_bag_data, l_bag_data, ratio):

    train_subspaces = []
    
    for i in range(IDs):
        images = []
        for j in range(round(N*ratio)):
            if l_bag_data[j] == i+1:
                images.append(X_bag_data[j])
        train_subspaces.append(images)
    return np.asarray(train_subspaces)

In [6]:
def SWSB(bag_subspaces, X_bar_bag):
    elements = np.asarray(bag_subspaces[0])
    elements_mean = np.mean(elements, axis=0)
    elements = np.subtract(elements,elements_mean)
    S_W = np.matmul(elements.T,elements)
    mean_diff = np.atleast_2d(elements_mean)-X_bar_bag
#     print(mean_diff.shape)
    S_B = np.matmul(mean_diff.T, mean_diff)
    for i in range(1,IDs):
#         print(i)
        elements = np.asarray(bag_subspaces[i])
#         print(train_subspaces.shape)
        elements_mean = np.mean(elements, axis=0)
        elements = np.subtract(elements,elements_mean)
        S_W += np.matmul(elements.T,elements)
        mean_diff = np.atleast_2d(elements_mean)-X_bar_bag
        S_B += np.matmul(mean_diff.T, mean_diff)
    
    return S_W, S_B
                   

In [7]:
#PCA
def PCA(X_train_pca, X_bar_pca):
    A = np.subtract(X_train_pca,X_bar_pca).T
#     print(A.shape)
#     print(np.matmul(A.T,A).shape)
    w_pca, v_pca = np.linalg.eigh((1/N)*np.matmul(A.T,A))
    inds = w_pca.argsort()[::-1]
    w_pca = w_pca[inds]
    v_pca = v_pca[:,inds]
    cum_w = np.cumsum(w_pca)/np.sum(w_pca);
    M_pca = np.argmax(cum_w>M_pca_ratio)
    w_pca = w_pca[:M_pca]
    v_pca = v_pca[:,:M_pca]
    v_pca = preprocessing.normalize(np.matmul(A,v_pca), axis=0)
    return w_pca, v_pca

In [8]:
#LDA
def LDA(S_W_bag, S_B_bag, v_pca):
    proj_1 = np.dot(np.dot(v_pca.T,S_W_bag),v_pca)
    proj_2 = np.dot(np.dot(v_pca.T,S_B_bag),v_pca)

    w_fld, v_fld = np.linalg.eigh(np.linalg.inv(proj_1).dot(proj_2))
    
    cum_w = np.cumsum(w_fld)/np.sum(w_fld);
    M_lda = 30
    
    inds = w_fld.argsort()[::-1]
    w_fld = w_fld[inds]
#     print(w_fld.shape)
    v_fld = v_fld[:,inds]

    
    w_fld = w_fld[:M_lda]
    v_fld = v_fld[:,:M_lda]
    return w_fld, v_fld
    
    

In [9]:
#W_OPT
def w_opt(v_pca, v_fld): 
    v_opt = np.dot(v_pca,v_fld)
    v_opt = preprocessing.normalize(v_opt,axis=0)
    return v_opt

In [10]:
#Get error of each bag
def error(v_opt, X_train_dataset, X_test_data, l_train_error, l_test_error):
    Y_train = []
    for i in range(int(N_bag)):
        Y_train.append(np.matmul(v_opt.T,X_train_dataset[i]))
        
    correct = 0
    l_pred = []
    
    for i in range(len(X_test)):
        y_test = np.matmul(v_opt.T, X_test_data[i])
        diff = np.subtract(Y_train, y_test)
        pred = np.diag(np.matmul(diff, diff.T))
        value = np.argmin(pred)
        l_pred.append(l_train_error[value])
        if l_train_error[value] == l_test_error[0][i]:
            correct += 1
            
    return correct/len(l_test[0]), l_pred

#### Calling Functions 
***For loop***
- make bags
- make training subspaces
- make eigen subspaces & S_W & S_B 
- PCA 
- LDA 
- W_opt 
- Error calculation
- Committie machine


In [11]:
# TEST SCRIPT
X_bag_0, l_bag_0, X_bar_0 = make_bag(X_train, l_train)

print('    X bag shape', X_bag_0.shape,'l bag shape', l_bag_0.shape, 'X bar bag', X_bar_0.shape)

bag_0_train_space = bag_train_space(X_bag_0, l_bag_0, bag_ratio)

print('    bag train space shape:', bag_0_train_space.shape)

S_W_0, S_B_0= SWSB(bag_0_train_space, X_bar_0)

print('    S_W', S_W_0.shape, 'S_B', S_B_0.shape)

    X bag shape (312, 2576) l bag shape (312,) X bar bag (2576,)
    bag train space shape: (52, 6, 2576)
    S_W (2576, 2576) S_B (2576, 2576)


In [12]:
print(l_bag_0.shape)

(312,)


In [13]:
def N_bag_error(No_bags, X_train, l_train):
    bag_error = []
    bag_pred = []
    for i in range(No_bags):
        print('    Bag:', i+1)
        X_bag, l_bag, X_bar_bag = make_bag(X_train, l_train)
        bag_space = bag_train_space(X_bag, l_bag, bag_ratio)
#         print(X_bar_bag.shape)
        S_W, S_B = SWSB(bag_space, X_bar_bag)
        w_pca, v_pca = PCA(X_bag, X_bar_bag)
        w_fld, v_fld = LDA(S_W, S_B, v_pca)
        v_opt = w_opt(v_pca, v_fld)
        err, l_pred = error(v_opt, X_bag, X_test, l_bag, l_test)
        print('    ', err)
        bag_error.append(err)
        bag_pred.append(l_pred)
        
        
    return bag_error, bag_pred      

In [14]:
# def commit_mac(bag_error, bag_pred):
#     for 
#     y_com = 

In [15]:
bags_error, bags_pred = N_bag_error(60, X_train, l_train)

    Bag: 1
     0.5961538461538461
    Bag: 2
     0.5673076923076923
    Bag: 3
     0.5769230769230769
    Bag: 4
     0.6153846153846154
    Bag: 5
     0.6057692307692307
    Bag: 6
     0.5769230769230769
    Bag: 7
     0.6057692307692307
    Bag: 8
     0.5961538461538461
    Bag: 9
     0.5384615384615384
    Bag: 10
     0.5480769230769231
    Bag: 11
     0.4519230769230769
    Bag: 12
     0.5480769230769231
    Bag: 13
     0.5576923076923077
    Bag: 14
     0.5480769230769231
    Bag: 15
     0.5769230769230769
    Bag: 16
     0.6346153846153846
    Bag: 17
     0.5769230769230769
    Bag: 18
     0.5576923076923077
    Bag: 19
     0.5576923076923077
    Bag: 20
     0.6442307692307693
    Bag: 21
     0.5096153846153846
    Bag: 22
     0.5865384615384616
    Bag: 23
     0.625
    Bag: 24
     0.5673076923076923
    Bag: 25
     0.5576923076923077
    Bag: 26
     0.5673076923076923
    Bag: 27
     0.5192307692307693
    Bag: 28
     0.5384615384615384
    Bag: 29
  

In [16]:
bags_pred = np.asarray(bags_pred)
print(bags_pred.shape)

(60, 104)


In [17]:
def commit_cor(bags_pred, l_test):
    correct = 0
    for i in range(len(bags_pred)):
#     for i in range(bags_pred.shape[0]):
#         value = np.floor(np.argmin(bags_pred[i])/round(8*bag_ratio))+1
# #         print(value, l_test[0][i])
        if bags_pred[i] == l_test[0][i]:
            correct += 1
    return correct/104

In [18]:
def commit_mac(bags_pred, bags_err):
    
    y_pred = []
    for i in range(bags_pred.T.shape[0]):
        im_pred = np.zeros(IDs)
        for j in range(bags_pred.T.shape[1]):
            im_pred[bags_pred.T[i][j]-1] += 1
        y_pred.append(np.argmax(im_pred)+1)
    return y_pred
    

In [19]:
bags_commit = commit_mac(bags_pred, bags_error)

In [20]:
print(commit_cor(bags_commit, l_test))

0.7403846153846154
